In [70]:
import openpyxl
import os

In [51]:
answer_key = "C:\\Users\\jetle\\Documents\\resource\\workbook\\answer_key.xlsx"
student_answers = "C:\\Users\\jetle\\Documents\\resource\\workbook\\answers.xlsx"

In [77]:
class KeyWord:
    #initilizing parent class constructor and a  a list for storing keyword
    def __init__(self,student_answer_path,answer_key_path):
        self.file_path = student_answer_path
        self._PATH = answer_key_path
        self.META_extension = ""
        self.keylist = [] #list to store keyword
        self.anslist = [] #list to store answer pieces
        self.frequencyTable = {}
        self.hitCount = 0
        self.parseChar = [',','.','!','“','”','(',')','{','}','[',']','"',"'"]     
        self.raw_data = []
        self.raw_answer = []
        self.question_dict = {} #storing question code and question string
        self.question_dict_student = {} #storing question code and corresponding student
        self.answer_dict = {} #storing question code and corresponding correct answer
        self.answer_dict_student = {} #storing question code and corresponding student answer
        self.mark_dict = {} #storing question code and corresponding mark
        self.mark_dict_student = {} #storing student code and mark
        self.key_dict = {} #storing the question code and corresponding keywords
        self.validate()
        self.parseFile()

    #function to check the extension of file
    def validate(self):
        split = os.path.splitext(self.file_path);
        extension = split[-1]
        if extension == ".txt":
            self.META_extension = "TEXT"
        elif extension == ".xlsx":
            self.META_extension = "EXCEL"
        else:
            self.META_extension = "incompatible"
        
    #function to manually insert a keyword
    def insertKey(self):
        limit = int(input("Enter the number of keywords : "))
        print("Enter Keywords")
        for i in range(limit):
            keyvalue = input()
            self.keylist.append(keyvalue)

    #function to read keyword from a text file.
    def insertKey(self):
        file = open(self._PATH, "rt",encoding="utf-8")
        for line in file:
            for key in line.split(","):
                self.keylist.append(key)
                
    #function to clean data from answer key
    #answer_key path is saved in _PATH file
    def cleanAnswerKey(self):
        answer_key_wb = openpyxl.load_workbook(self._PATH)
        #selecting the apporpriate sheet
        answer_key_sheet = answer_key_wb['answer_key']
        #iteration via answer sheet
        for i in range(2, answer_key_sheet.max_row+1):
            for j in range(1, answer_key_sheet.max_column+1):
                cell_obj = answer_key_sheet.cell(row=i, column=j)
                self.raw_data.append(cell_obj.value)
                
        for i in range(0,len(self.raw_data),5):
            self.question_dict[self.raw_data[i]] = self.raw_data[i+1]
            self.answer_dict[self.raw_data[i]] = self.raw_data[i+2]
            self.mark_dict[self.raw_data[i]] = self.raw_data[i+4]
            self.key_dict[self.raw_data[i]] = self.raw_data[i+3]
    
    #function to clean data form student answer key
    #file path is store in file_path
    def cleanStudentAnswer(self):
        student_answer_wb = openpyxl.load_workbook(self.file_path)
        #selecting active sheet
        student_answer_sheet = student_answer_wb.active
        for i in range(2,student_answer_sheet.max_row+1):
            for j in range(1,student_answer_sheet.max_column+1):
                cell_obj = student_answer_sheet.cell(row=i,column=j)
                self.raw_answer.append(cell_obj.value)
        
        for i in range(0,len(self.raw_answer),5):
            self.question_dict_student[self.raw_answer[i+1]] = self.raw_answer[i]
            self.answer_dict_student[self.raw_answer[i+1]] = self.raw_answer[i+3]
            self.mark_dict_student[self.raw_answer[i+1]] = self.raw_answer[i+4]
                

    #file to be managed by its extension 
    def parseFile(self):
        if self.META_extension == "TEXT":
            self.parseTextFile()
        elif self.META_extension == "EXCEL":
            self.cleanAnswerKey()
            self.cleanStudentAnswer()
            self.hitCounter(self.key_dict,self.answer_dict)
            
            
    #method to calculate the mark from hit ratio
    def markCalculator(self,i):
        try:
            keyCount = len(self.keylist)
            percentage = (self.hitCount/keyCount)*100
            #self.mark_dict_student[i] = (percentage/100)*self.mark_dict[i]
            print(self.mark_dict_student[i], " ",(percentage/100)*self.mark_dict[i])
        except:
            #self.mark_dict_student[i] = 0
            print("Exception")
        
    #method to reset the values
    def reset(self):
        self.keylist.clear()
        self.frequencyTable.clear()
        self.hitCount = 0
        

    #function to generate the frequency of keywords in the given file
    def parseTextFile(self):
        with open(self.file_path, 'r', encoding="utf-8") as fp:
            for line in fp:
                for word in line.split():
                    #parsing the word for potential mismatches
                    for char in self.parseChar:
                        word = word.rstrip(char) #right side stripping
                        word = word.lstrip(char) #left side stripping
                    if word in self.keylist:
                        if word in self.frequencyTable:
                            self.frequencyTable[word] += 1
                        else:
                            self.hitCount += 1
                            self.frequencyTable[word] = 1
                            
    #function to detect hit counter
    def hitCounter(self,keyword,answer):
        for i in keyword:
            for j in answer:
                if i == j:
                    self.keylist = keyword[i].split(",")
                    self.anslist = answer[i].split()
                    for word in self.anslist:
                        for char in self.parseChar:
                            word = word.rstrip(char) #right side stripping
                            word = word.lstrip(char) #left side stripping
                        if word in self.keylist:
                            if word in self.frequencyTable:
                                self.frequencyTable[word] += 1
                            else:
                                self.hitCount += 1
                                self.frequencyTable[word] = 1   
                self.markCalculator(i)
                self.reset()
    

        

In [78]:
K = KeyWord(student_answers,answer_key)


0   3.0952380952380953
Exception
Exception
0   4.722222222222222
